# Planification de la tournée d’une infirmière.


Une infirmière doit s’occuper de plusieurs patients à domicile. Pour lui éviter des frais importants et un temps de travail allongé, on cherche à réduire son temps de trajet. Il s’agit donc de décider à l’avance l’ordre dans lesquels elle visitera ses patients. Nous faisons ici quelques hypothèses de travail, hypothèses qui pourront être modifiées en fonction de l’avancement des travaux :

- aucun patient n’est prioritaire ;
- le temps de trajet est proportionnel à la distance parcourue, la constante de proportionnalité ne dépend pas du moment de la journée (il n’y a pas de bouchons !) ;
- les patients restent chez eux toute la journée, et sont disponibles pour les soins toute la journée ;
- l’infirmière débute une tournée en partant de chez elle et la termine en rentrant chez elle. Les trajets aller et retour doivent bien sûr être comptés.

Les adresses des patients seront fournies sous la forme de deux coordonnées : la latitude et la longitude (des angles en degrés), la distance entre les points $A$ et $B$ de longitudes $\lambda_A$ et $\lambda_B$ respectivement et de latitude $\phi_A$ et $\phi_B$ peut être calculée en première approximation (théorème de Pythagore…) par la formule :
$$d = 1.852 \times 60 \times \sqrt{(\phi_B - \phi_A)^2 + (\lambda_B - \lambda_A)^2 \cos\left(\dfrac{\phi_A + \phi_B}{2}\right)^2}.$$

d’après [ce site](http://villemin.gerard.free.fr/aGeograp/Distance.htm).

## Algorithmes pour planifier la tournée

Nous proposons plusieurs algorithmes de planification :
- trajet aléatoire amélioré petit à petit
- approche gloutonne.

Ces algorithmes sont détaillés dans la suite. Dans tous les cas, les fonctions utilisées prendront en argument un dictionnaire dont les clés sont des identifiants des patients, et les valeurs associées leurs coordonnées sous la forme (latitude, longitude). Les coordonnées de l’infirmière se trouvent aussi dans ce dictionnaire, sous la clé "infirmière". Les autres clés correspondent aux patients.

In [18]:
Identifiant = str
Latitude = float
Longitude = float
Données = dict[Identifiant, tuple[Latitude, Longitude]]
Trajet = list[Identifiant]

INFIRMIERE = "infirmière"

### Trajet aléatoire

Le langage Python permet très facilement de générer des trajets aléatoires. La bibliothèque qui le permet se nomme `random`. On l’importe :

In [7]:
import random as rd

Compléter la fonction `trajet_aléatoire` suivante pour qu’elle retourne un trajet aléatoire. Elle prend en argument les coordonnées des patients sous la forme précisée précédemment et retourne la liste des identifiants dans un ordre aléatoire. Attention, aucun patient ne doit être oublié ni visité deux fois. Un trajet doit commencer et finir par `INFIRMIERE`.

In [19]:
def trajet_aléatoire(données: Données) -> Trajet:
    patients = [p for p in données.keys() if p != INFIRMIERE]
    patients.shuffle()
    return [INFIRMIERE] + patients + [INFIRMIERE]

On est en droit de douter de l’efficacité d’un trajet aléatoire ! L’idée est plutôt que cet algorithme fournit une première solution que l’on va chercher à améliorer. Une possibilité est d’échanger deux patients : si la distance totale parcourue est inférieure, on garde cette nouvelle solution, sinon on la rejette. On effectue ces échanges pendant une durée déterminée. On procède en plusieurs étapes.

Compléter la fonction `distance` qui prend en argument les données géographiques des patients ainsi que les identifiants de deux patients et qui retourne la distance entre ces deux patients.

In [20]:
import numpy as np

def distance(données: Données, source: Identifiant, cible: Identifiant) -> float:
    (lambda_a, phi_a) = données[source]
    (lambda_b, phi_b) = données[cible]
    x = phi_b - phi_a
    y = (lambda_b - lambda_a) * np.cos((phi_a + phi_b)/2)
    return 1.852 * 60 * np.sqrt(x**2 + y**2)

Compléter la fonction `distance_totale`, qui prend en argument les données géographiques des patients ainsi qu’un trajet, et qui retourne la distance parcourue.

In [21]:
def distance_totale(données: Données, trajet: Trajet) -> float:
    n = len(trajet)
    t = 0
    for i in range(n-1):
        t += distance(données, trajet[i], trajet[i+1])
    return t

Compléter la fonction `échange`, qui prend en argument un trajet, et produit un autre trajet en échangeant aléatoirement deux patients. Attention un trajet doit commencer et terminer par `INFIRMIERE`.

In [22]:
def échange(trajet: Trajet) -> Trajet:
    n = len(trajet-2)
    a,b = rd.choices(range(1,n), k=2)
    return trajet[:a]+[trajet[b]]+trajet[a+1:b]+[trajet[a]]+trajet[b+1:]

Compléter la fonction `trajet_aléatoire_amélioré`, qui prend en argument les données géographiques des patients et une durée maximale en seconde, qui retourne un trajet, en améliorant un trajet aléatoire par échange aléatoire de deux patients, en limitant la durée d’exécution.

In [23]:
from datetime import datetime

def trajet_aléatoire_amélioré(données: Données, temps_max: int) -> Trajet:
    maintenant = datetime.now()
    trajet = trajet_aléatoire(données)
    while time.now()-maintenant < temps_max:
        t = échange(trajet)
        if distance_totale(t) < distance(trajet):
            trajet = t
    return trajet

En rejetant les mauvaises solutions, il se peut qu’on rate une excellente solution (problème de l’optimum local). Une approche intéressante est celle dite du [recuit simulé](https://fr.wikipedia.org/wiki/Recuit_simul%C3%A9) : si une mauvaise solution apparaît, on décide de la garder avec une certaine probabilité, probabilité qui décroit avec le temps.

In [24]:
# TODO

### Approche gloutonne

Plutôt que de laisser faire le hasard, on peut procéder de manière plus pragmatique : quand l’infirmière a fini avec un patient (ou quand elle démarre sa tournée), elle se dirige vers le patient non visité le plus proche d’elle. Cette solution est dite *gloutonne*. Malheureusement on n’est pas assuré de trouver la meilleure solution.

Compléter la fonction `trajet_glouton` qui prend en argument les données géographiques des patients et qui retourne le trajet obtenu en faisant des choix gloutons.

In [25]:
def trajet_glouton(données: Données) -> Trajet:
    patients = [p for p in données.keys() if p != INFIRMIERE]
    trajet = [INFIRMIERE]
    position = INFIRMIERE
    while patients:
        p = min(patients, key = lambda q: distance(données, p, q))
        trajet.append(p)
        patients.remove(p)
    return trajet

## Tests

Pour vérifier les algorithmes précédents, il faut générer des données géographiques, plus ou moins conséquentes suivant que l’on veut tester les performances ou non de ces algorithmes. Il faut vérifier que les algorithmes 
- retournent des données en accord avec les contraintes posées (un trajet est une liste d’identifiants, qui commence et se termine par INFIRMIERE…)
- ne mettent pas trop de temps à s’exécuter
- fournissent des solutions raisonnables.

On peut générer des données de manière complètement aléatoire, ou chercher des données plus réalistes en consultant des statistiques officielles, ou encore des données destinées à «piéger» les algorithmes.

## Visualisation des résultats obtenus

L’équipe de tests a généré des instances et l’équipe de codage les algorithmes. Il faut maintenant présenter ces résultats, de manière graphique, en faisant apparaître les informations pertinentes, et en laissant la possibilité à l’utilisateur de régler différents paramètres utilisés dans le code. C’est là que vous intervenez. Vous utiliserez la bibliothèque `ipywidget`.